In [1]:
from matplotlib import pyplot as plt 
import numpy as np 
import pandas as pd 
import plotly.express as px
import plotly.io as pio
import plotly.graph_objs as go
import plotly.offline as pyo
from plotly.subplots import make_subplots

import pickle

import collections
from collections import Counter

from matplotlib import rc

## Load pikle

In [2]:
read_input = open('data/EDADS_subtype_timelines_agecorrected_opt.pickle','rb')
load_inputs = pickle.load(read_input)
read_input.close()

T, S, Sboot = load_inputs

diagnosis = np.load('data/diagnosis.npy', allow_pickle=True)

/Users/macos/opt/anaconda3/envs/umc/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning:

Trying to unpickle estimator MinCovDet from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations



## Pie-plot function

In [48]:
def piechart_multiple(S, diagnosis, subtype_labels=None, chosen_subtypes = None):
    """
    Creates a pie chart of subtypes in the data   
    :param S: subtyping dictionary, subtypes for each patient individually
    :param diagnosis: np.array or list with diagnosis labels corresponding to records in S
    :param subtype_lables: a list with names of the subtype labels (optional)
    :param chosen_subtypes: a list with diagnosis labels to consider in the plot
    :return: plotly Pie Figure
    """
    
    # Get subtype labels
    unique_subtypes = np.unique(S['subtypes'][~np.isnan(S['subtypes'])])
    if subtype_labels is None:
        subtype_labels = []
        for i in range(len(unique_subtypes)):
            subtype_labels.append('Subtype '+str(int(unique_subtypes[i])))
            
    # Set colors
    default_colors = ['#d2fafc','#62a2f5','#66cfff','#0e4287','#5894ed','#1217a3']
    color_map = {subtype_labels[i] : default_colors[i] for i in range(len(subtype_labels))}
    color_map['Outliers']='#8e9191'
    print(color_map)
            
    subtype_labels.append("Outliers")
    
            
    subtypes = list(S['subtypes'])
    subtypes = ['Outliers' if np.isnan(s) else s for s in subtypes]
    
    # Get diagnosis lables (exclude controls)
    diagnosis_labels = list(set(diagnosis))
    diagnosis_labels.remove('CN')
    diagnosis_labels.sort()
    
    if chosen_subtypes is None:
        chosen_subtypes=diagnosis_labels

    df = {'Diagnosis':diagnosis, 'Subtype':subtypes}
    df = pd.DataFrame(df)
    df = df[df['Diagnosis']!='CN']
        
    # Create DataFrame for quering to plotting
    df_subtypes = subtype_labels*len(diagnosis_labels)
    df_diagnosis = [d for d in diagnosis_labels for i in range(len(subtype_labels))]
    counts = list(df.groupby(['Diagnosis','Subtype']).size())
    df = pd.DataFrame({'Diagnosis':df_diagnosis, 'Subtype':df_subtypes, 'Counts': counts})

    # Query diagnosis for plotting
    df_plot = df[df['Diagnosis'].isin(chosen_subtypes)]

    fig = px.pie(df_plot, values='Counts', names='Subtype', title='Subtypes',color='Subtype',
                color_discrete_map=color_map)

    # style the plot
    fig.update_traces(textposition='inside', textinfo='value+percent')   
    
    fig.update_layout(title='',
                  title_x=0.5,
                  title_font_size=24,
                  legend_font_size=18)

    return fig

In [49]:
# FOR TESTING
chosen_subtypes = ['AD', 'MCI']

In [50]:
fig = piechart_multiple(S=S,
                        diagnosis=diagnosis,
                      chosen_subtypes=chosen_subtypes)
fig

{'Subtype 0': '#d2fafc', 'Subtype 1': '#62a2f5', 'Subtype 2': '#66cfff', 'Subtype 3': '#0e4287', 'Outliers': '#8e9191'}


In [13]:
color_discrete_map={'Thur':'lightcyan','Fri':'cyan','Sat':'royalblue','Sun':'darkblue'}
color_discrete_map

{'Thur': 'lightcyan', 'Fri': 'cyan', 'Sat': 'royalblue', 'Sun': 'darkblue'}

In [14]:
default_colors = ['#d2fafc','#8dcbd9','#66cfff','#317fa3','#5894ed','#1217a3']

In [21]:
unique_subtypes = np.unique(S['subtypes'][~np.isnan(S['subtypes'])])
subtype_labels = []
for i in range(len(unique_subtypes)):
    subtype_labels.append('Subtype '+str(int(unique_subtypes[i])))

color_map = {subtype_labels[i] : default_colors[i] for i in range(len(subtype_labels))}

# #8e9191

In [22]:
color_map['Outliers']='#8e9191'

In [23]:
color_map

{'Subtype 0': '#d2fafc',
 'Subtype 1': '#8dcbd9',
 'Subtype 2': '#66cfff',
 'Subtype 3': '#317fa3',
 'Outliers': '#8e9191'}